In [2]:
##
import stanza
from typing import List
from stanza_batch import batch
import json
from nltk.corpus import stopwords

## load necessary processors
nlp = stanza.Pipeline('en', processors='lemma,tokenize,pos,depparse',\
    depparse_model_path='../../stanza_model/en_outall_parser.pt')

## read input file
target_test=open('../dataset/clean/commongen.test.src_alpha.txt')
# target_test=open('/home/mbastan/context_home/structuralDecoding/neurologic_decoding/SuMe/commongen.sume_dev_original.tgt.txt')

lines = target_test.readlines()
target_test.close()


    
import pickle
try:
    with open('stanza_documents_commongen_testt.pk','rb') as f:
        stanza_documents=pickle.load(f)
except:
    ## batch document processing to increase the speed
    stanza_documents: List[stanza.Document] = []
    for document in batch(lines, nlp, batch_size=64): # Default batch size is 32
        stanza_documents.append(document)
    ## save the documents on a file
    with open('stanza_documents_commongen_test.pk','wb') as f:
        pickle.dump(stanza_documents,f)   
                

/home/mbastan/anaconda3/envs/hug3/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-20 05:49:10 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-01-20 05:49:10 INFO: Loading these models for language: en (English):
| Processor | Package                 |
---------------------------------------
| tokenize  | combined                |
| pos       | combined                |
| lemma     | combined                |
| depparse  | ../../stan..._parser.pt |

2023-01-20 05:49:10 INFO: Use device: gpu
2023-01-20 05:49:10 INFO: Loading: tokenize
2023-01-20 05:49:12 INFO: Loading: pos
2023-01-20 05:49:12 INFO: Loading: lemma
2023-01-20 05:49:13 INFO: Loading: de

In [9]:


with open('../dataset/clean/constraint/commongen_test_all.constraint.json','w') as f:
            for doc in  stanza_documents:
                sent = doc.sentences[0]
                arr_all = []
                for word in sent.words:
                    if word.upos == "NOUN":
                        arr_all.append([word.lemma,'nsubj','subj','obj','obl'])
                    elif word.upos == "VERB":
                        arr_all.append([word.lemma,'root'])
                        
                ## write all structures
                json.dump(arr_all,f)
                f.write('\n')
                
                


: 

In [28]:
import json

with open('../dataset/clean/commongen.test.cs_str.txt') as f:
    word_lines=f.readlines()
with open('../dataset/clean/constraint/test_all.constraint.json','w') as f:
    for jj, line in  enumerate(word_lines):
        words=line.strip().split('#')
        arr_all=[]
        sent = stanza_documents[jj].sentences[0]
        word_lemma_map={}
        for word in sent.words:
            word_lemma_map[word.text]=word.lemma
        assert len(words) == len(sent.words)
        for ind,word in enumerate(words):
            lemma = word_lemma_map[word[:-2]]
            
            if word.endswith('_N') :
                arr_all.append([lemma,'nsubj','obj','obl'])
            elif word.endswith('_V'):
                arr_all.append([lemma,'root'])
            else:
                print('error darim word is',word)
                # break
                
        ## write all structures
        json.dump(arr_all,f)
        f.write('\n')
                
                


In [29]:
arr_all

[['eye', 'nsubj', 'obj', 'obl'],
 ['hang', 'root'],
 ['head', 'nsubj', 'obj', 'obl'],
 ['shut', 'root'],
 ['squeeze', 'root']]

In [30]:
words

['eye_N', 'hang_V', 'head_N', 'shut_V', 'squeeze_V']